In [1]:
import numpy as np
import pandas as pd
import pathlib
import sys

# Absolute path to this file
MY_PAPI_DIR = pathlib.Path().absolute()
# Now, we have to move to the root of this workspace ([prev. path]/TFG)
MY_PAPI_DIR = MY_PAPI_DIR.parent.parent.parent.parent.absolute()
# From the root (TFG/) access to my_papi dir. and its content
MY_PAPI_DIR = MY_PAPI_DIR / "my_papi"
# Folder where the configuration files are located
CFG_DIR = MY_PAPI_DIR / "conf"
# Folder where the library is located
LIB_DIR = MY_PAPI_DIR / "lib"
# Folder where the source codes are located
SRC_DIR = MY_PAPI_DIR / "src"

# Add the source path and import the library
sys.path.insert(0, str(SRC_DIR))
from MyPapi import *

In [76]:
# Time to print the values of the measure M1
#pd.set_option('display.width', 200)

# Name of csv file
csv_file = "/home/jlpadillas01/TFG/tests/tensorflow/mnist/out/mnist_train_papi.csv"

# Read csv with the following name of columns
df1 = pd.read_csv(csv_file, header=None, sep=":", names=["CPU", "Value", "Unit", "Event Name"])

# Get the rows which have no 0 values
df1 = df1[df1.Value != 0]

# Calculate the average from the rows which have the same Event name
df1['Avg'] = df1.groupby('Event Name')['Value'].transform('mean')
df1

,CPU,Value,Unit,Event Name,Avg
0,2,193532612763,NaN,PERF_COUNT_HW_INSTRUCTIONS,1.926831e+11
1,2,16637978284,NaN,PERF_COUNT_HW_BRANCH_INSTRUCTIONS,1.652379e+10
2,2,67197413612,NaN,L1-DCACHE-LOADS,6.695347e+10
3,2,15638234031,NaN,L1-DCACHE-STORES,1.545256e+10
4,2,247316414,NaN,fp_arith_inst_retired.scalar_double,2.472346e+08
5,2,154688753,NaN,fp_arith_inst_retired.scalar_single,1.546888e+08
6,2,15056280,NaN,fp_arith_inst_retired.128b_packed_single,1.505628e+07
7,2,3403140,NaN,fp_arith_inst_retired.256b_packed_single,3.403140e+06
8,2,122693346870,NaN,fp_arith_inst_retired.512b_packed_single,1.226933e+11
270,2,191722933694,NaN,PERF_COUNT_HW_INSTRUCTIONS,1.926831e+11


In [77]:
# Time to print the values of the measure M1

# Name of csv file
csv_file = "/home/jlpadillas01/TFG/tests/tensorflow/mnist/out/mnist_train_callback.csv"

# Read csv with the following name of columns
df2 = pd.read_csv(csv_file, header=None, sep=":", names=["CPU", "Value", "Unit", "Event Name"])

# Get the rows which have no 0 values
df2 = df2[df2.Value != 0]

# Calculate the average from the rows which have the same Event name
df2['Avg'] = df2.groupby('Event Name')['Value'].transform('mean')
df2

,CPU,Value,Unit,Event Name,Avg
0,2,193348191211,NaN,PERF_COUNT_HW_INSTRUCTIONS,1.929248e+11
1,2,16622251581,NaN,PERF_COUNT_HW_BRANCH_INSTRUCTIONS,1.656340e+10
2,2,67139940133,NaN,L1-DCACHE-LOADS,6.702425e+10
3,2,15615410143,NaN,L1-DCACHE-STORES,1.553569e+10
4,2,247318651,NaN,fp_arith_inst_retired.scalar_double,2.472812e+08
5,2,154688753,NaN,fp_arith_inst_retired.scalar_single,1.546888e+08
6,2,15056280,NaN,fp_arith_inst_retired.128b_packed_single,1.505628e+07
7,2,3403140,NaN,fp_arith_inst_retired.256b_packed_single,3.403140e+06
8,2,122693346870,NaN,fp_arith_inst_retired.512b_packed_single,1.226933e+11
270,2,191548125053,NaN,PERF_COUNT_HW_INSTRUCTIONS,1.929248e+11


In [75]:
# Time to print the values of the measure M2

# Name of csv file
csv_file = "/home/jlpadillas01/TFG/tests/tensorflow/mnist/out/mnist_train_each_epoch.csv"

# Read csv with the following name of columns
df3 = pd.read_csv(csv_file, header=None, sep=":", names=["CPU", "Value", "Unit", "Event Name"])

# Get the rows which have no empty values
df3 = df3[df3.Value != 0]

# Define the number of epochs and number of measures
num_epochs = 30
num_measures = 5

# Get the events measured
events = df3["Event Name"].unique()

# Get the len of the new Dataframe
size = len(df3.index)

# Split the Dataframe in num_measures
arrs = np.array_split(df3, num_measures)

# Store that mean columns in a new Dataframe
data = []
headers = []
i = 0
# Calculate the mean of each iteration
for arr in arrs:
    arr = arr.reset_index(drop=True)
    arr['Avg'] = arr.groupby('Event Name')['Value'].transform('mean')
    i = i + 1
    headers.append("Measure_" + str(i))
    data.append(arr['Avg'].head(len(events)))

# Creates a new df from the avg of the iters
df3 = pd.concat(data, axis=1, keys=headers)

# Add the events names
df3.insert(0, 'Event Name', events)

# Creates a new avg column from all the measures
df3['Avg'] = df3.mean(axis=1)

df3

,Event Name,Measure_1,Measure_2,Measure_3,Measure_4,Measure_5,Avg
0,PERF_COUNT_HW_INSTRUCTIONS,6.433955e+09,6.431052e+09,6.424974e+09,6.429421e+09,6.385306e+09,6.420942e+09
1,PERF_COUNT_HW_BRANCH_INSTRUCTIONS,5.523381e+08,5.512157e+08,5.501345e+08,5.510285e+08,5.472687e+08,5.503971e+08
2,L1-DCACHE-LOADS,2.235538e+09,2.235970e+09,2.233437e+09,2.235186e+09,2.220496e+09,2.232125e+09
3,L1-DCACHE-STORES,5.193532e+08,5.197940e+08,5.186852e+08,5.197724e+08,5.056542e+08,5.166518e+08
4,fp_arith_inst_retired.scalar_double,8.244126e+06,8.243732e+06,8.237161e+06,8.243882e+06,8.236990e+06,8.241178e+06
5,fp_arith_inst_retired.scalar_single,5.156290e+06,5.156290e+06,5.156290e+06,5.156290e+06,5.156290e+06,5.156290e+06
6,fp_arith_inst_retired.128b_packed_single,5.018760e+05,5.018760e+05,5.018760e+05,5.018760e+05,5.018760e+05,5.018760e+05
7,fp_arith_inst_retired.256b_packed_single,1.134380e+05,1.134380e+05,1.134380e+05,1.134380e+05,1.134380e+05,1.134380e+05
8,fp_arith_inst_retired.512b_packed_single,4.089778e+09,4.089778e+09,4.089778e+09,4.089778e+09,4.089778e+09,4.089778e+09


In [29]:
# Join the two dataframes in another one
data = [df1["Avg"], df2["Avg"]]
headers = ["df1", "df2"]
df3 = pd.concat(data, axis=1, keys=headers)
df3['Value_diff'] = (df3['df1'] - df3['df2']).abs()
#df3

In [30]:
with pd.ExcelWriter('output.xlsx') as writer:
    df1.to_excel(writer, sheet_name='mnist_train_papi')
    df2.to_excel(writer, sheet_name='mnist_train_callback')
    df3.to_excel(writer, sheet_name='mnit_M1_comparation')

In [ ]:
# Time to print the values of the measure M2

# Name of csv file
csv_file = "/home/jlpadillas01/TFG/tests/tensorflow/mnist/out/mnist_train_each_batch.csv"

# Read csv with the following name of columns
df4 = pd.read_csv(csv_file, header=None, sep=":", names=["CPU", "Value", "Unit", "Event Name"])

# Get the rows which have no empty values
df4 = df4[df4.Value != 0]

# Define the number of epochs and number of measures
num_epochs = 30
num_measures = 5

# Get the events measured
events = df4["Event Name"].unique()

# Get the len of the new Dataframe
size = len(df4.index)

# Split the Dataframe in num_measures
arrs = np.array_split(df4, num_measures)

# Store that mean columns in a new Dataframe
data = []
headers = []
i = 0
# Calculate the mean of each iteration
for arr in arrs:
    arr = arr.reset_index(drop=True)
    arr['Avg'] = arr.groupby('Event Name')['Value'].transform('mean')
    i = i + 1
    headers.append("Measure_" + str(i))
    data.append(arr['Avg'].head(len(events)))

# Creates a new df from the avg of the iters
df5 = pd.concat(data, axis=1, keys=headers)

# Add the events names
df5.insert(0, 'Event Name', events)

# Creates a new avg column from all the measures
df5['Avg'] = df5.mean(axis=1)

df5